In [1]:
import sys
sys.path.append('..')  # Make sure project root is in the path

from sft import do_sample

In [2]:
from sft import load_tokenizer_model, load_dataset

device = 'cuda'
tokenizer, model = load_tokenizer_model('../outputs/buddygpt-0.1b-base', 1024, device)

151669
BuddyGPTForCausalLM(
  (model): BuddyGPTModel(
    (embed_tokens): Embedding(151669, 768)
    (layers): ModuleList(
      (0-7): 8 x DecoderLayer(
        (self_attn): SdpaAttention(
          (q_proj): Linear(in_features=768, out_features=768, bias=True)
          (k_proj): Linear(in_features=768, out_features=384, bias=True)
          (v_proj): Linear(in_features=768, out_features=384, bias=True)
          (o_proj): Linear(in_features=768, out_features=768, bias=False)
          (rotary_emb): RotaryEmbedding()
        )
        (mlp): GateMLP(
          (gate_proj): Linear(in_features=768, out_features=1536, bias=False)
          (up_proj): Linear(in_features=768, out_features=1536, bias=False)
          (down_proj): Linear(in_features=1536, out_features=768, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): RMSNorm((768,), eps=1e-06, elementwise_affine=True)
        (post_layernorm): RMSNorm((768,), eps=1e-06, elementwise_affine=True)
      )
    )
 

In [3]:
ds = load_dataset(tokenizer, 30, 1024)
ds[:2]

Repo card metadata block was not found. Setting CardData to empty.


Map (num_proc=30):   0%|          | 0/1649399 [00:00<?, ? examples/s]

Dataset({
    features: ['text'],
    num_rows: 1649399
})


{'text': ['<|im_start|>user\n自然语言推理：\n前提：家里人心甘情愿地养他,还有几家想让他做女婿的\n假设：他是被家里人收养的孤儿<|im_end|>\n<|im_start|>assistant\n<think>\n\n</think>\n\n中立<|im_end|>\n',
  '<|im_start|>user\n在上海的苹果代工厂，较低的基本工资让工人们形成了“软强制”的加班默契。加班能多拿两三千，“自愿”加班成为常态。律师提示，加班后虽能获得一时不错的报酬，但过重的工作负荷会透支身体，可能对今后劳动权利造成不利影响。\n输出摘要：<|im_end|>\n<|im_start|>assistant\n<think>\n\n</think>\n\n苹果代工厂员工调查：为何争着“自愿”加班<|im_end|>\n']}

In [4]:
print(ds[3]['text'])

<|im_start|>user
生成音乐热评<|im_end|>
<|im_start|>assistant
<think>

</think>

1、感謝大家支持！然後就是Jane的人聲真的太好聽了，remix很自然就做出來了哈哈。為了讓人聲情感突出，這版本並沒有用太多搶耳的音色，只保留了一點點自己bass風格的律動感，然後我拿起一把Fender吉他就已經是主旋律了。Anyway , 超榮幸能參與這張專輯的！ Enjoy！
2、这首
有种退一步现世安稳，进一步天地广阔的感觉
好适合拿去当剪辑的BGM
在成长逐渐变得温柔又坚强的那种<|im_end|>



In [28]:

# 3. Tokenize 文本（重点）
encoded = tokenizer(ds[:1]['text'], return_tensors="pt")
encoded

{'input_ids': tensor([[151644,   8948,    198,   2610,    525,    264,  10950,  17847,     11,
           3465,    553,   3960,     17,    776,      0, 151645,    198, 151644,
            872,    198,  44063,  87752,  99534,  16744, 101047,  55338,   2073,
            285,    854,  27733,  99689, 109762,   2073,  16123,    854,   8997,
            785,  19145,    374,  17923,    323,    279,   4349,    374,   2238,
             13,    576,   9853,  12644,    374,  49584,    304,    279,   7015,
            323,    279,  29464,   1021,    374,  35918,     13, 151645,    198,
         151644,  77091,    198, 151667,    271, 151668,    271,    785,  19145,
            572,  17923,    323,    279,   4349,    572,   2238,     13,    576,
           9853,  12644,    572,  49584,    304,    279,   7015,    323,    279,
          29464,   1021,    572,  35918,     13, 151645,    198]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         

In [36]:
tokenizer.encode('hello<|im_end|>', add_special_tokens=False)

[14990, 151645]

In [30]:
from trl import SFTConfig, SFTTrainer, DataCollatorForCompletionOnlyLM
# text = '<|im_start|>system\nYou are a helpful assistant, created by learn2pro!<|im_end|>\n<|im_start|>user\n将以下短文中的所有“is”动词改为“was”。\nThe cake is delicious and the pie is too. The ice cream is melting in the sun and the lemonade is refreshing.<|im_end|>\n<|im_start|>assistant\n<think>\n\n</think>\n\nThe cake was delicious and the pie was too. The ice cream was melting in the sun and the lemonade was refreshing.<|im_end|>\n'

# sample = {
#     "text": "<|im_start|>user\n中国的首都是哪？<|im_end|>\n<|im_start|>assistant\n中国的首都是北京。<|im_end|>"
# }
sample = {
    "input_ids": [151644,   8948,    198,   2610,    525,    264,  10950,  17847,     11,
           3465,    553,   3960,     17,    776,      0, 151645,    198, 151644,
            872,    198,  44063,  87752,  99534,  16744, 101047,  55338,   2073,
            285,    854,  27733,  99689, 109762,   2073,  16123,    854,   8997,
            785,  19145,    374,  17923,    323,    279,   4349,    374,   2238,
             13,    576,   9853,  12644,    374,  49584,    304,    279,   7015,
            323,    279,  29464,   1021,    374,  35918,     13, 151645,    198,
         151644,  77091,    198, 151667,    271, 151668,    271,    785,  19145,
            572,  17923,    323,    279,   4349,    572,   2238,     13,    576,
           9853,  12644,    572,  49584,    304,    279,   7015,    323,    279,
          29464,   1021,    572,  35918,     13, 151645,    198]
}

collator = DataCollatorForCompletionOnlyLM(
    tokenizer=tokenizer,
    instruction_template="<|im_start|>user",  # 开始 loss 的位置
    response_template="<|im_start|>assistant",  # 如果你想从 assistant 开始一直算 loss，可以省略
    mlm=False,
)

batch = collator([sample])
batch

{'input_ids': tensor([[151644,   8948,    198,   2610,    525,    264,  10950,  17847,     11,
           3465,    553,   3960,     17,    776,      0, 151645,    198, 151644,
            872,    198,  44063,  87752,  99534,  16744, 101047,  55338,   2073,
            285,    854,  27733,  99689, 109762,   2073,  16123,    854,   8997,
            785,  19145,    374,  17923,    323,    279,   4349,    374,   2238,
             13,    576,   9853,  12644,    374,  49584,    304,    279,   7015,
            323,    279,  29464,   1021,    374,  35918,     13, 151645,    198,
         151644,  77091,    198, 151667,    271, 151668,    271,    785,  19145,
            572,  17923,    323,    279,   4349,    572,   2238,     13,    576,
           9853,  12644,    572,  49584,    304,    279,   7015,    323,    279,
          29464,   1021,    572,  35918,     13, 151645,    198]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         

In [31]:
for input_id, label in zip(batch['input_ids'][0], batch['labels'][0]):
    token = tokenizer.decode([input_id])
    print(f"{token:<10} | Label: {label}")

<|im_start|> | Label: -100
system     | Label: -100

          | Label: -100
You        | Label: -100
 are       | Label: -100
 a         | Label: -100
 helpful   | Label: -100
 assistant | Label: -100
,          | Label: -100
 created   | Label: -100
 by        | Label: -100
 learn     | Label: -100
2          | Label: -100
pro        | Label: -100
!          | Label: -100
<|im_end|> | Label: -100

          | Label: -100
<|im_start|> | Label: -100
user       | Label: -100

          | Label: -100
将          | Label: -100
以下         | Label: -100
短          | Label: -100
文          | Label: -100
中的         | Label: -100
所有         | Label: -100
“          | Label: -100
is         | Label: -100
”          | Label: -100
动          | Label: -100
词          | Label: -100
改为         | Label: -100
“          | Label: -100
was        | Label: -100
”          | Label: -100
。
         | Label: -100
The        | Label: -100
 cake      | Label: -100
 is        | Label: -100
 delicious | Label: -

In [ ]:
resp = do_sample(tokenizer, model, '中国首都是哪?')
print(resp)

In [ ]:
import sys
sys.path.append('..')

from model.utils import calc_token
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer
import model.modeling_tinyllm

model_id = 'learn2pro/buddygpt-0.2b-chat-zh'
tokenizer = AutoTokenizer.from_pretrained(model_id)



# ds = load_dataset("wangrui6/Zhihu-KOL", split="train")
ds = load_dataset("openbmb/Ultra-FineWeb", split="zh")
calc_token(ds, tokenizer, field='content')